In [1]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils import class_weight

# Check if GPU is available
print("GPU Available:", tf.config.list_physical_devices('GPU'))
# Dataset directory
base_dir = r"D:\dataset\dataset\New folder"  
img_size = (128, 128)  # Increased for better feature extraction
batch_size = 32
epochs = 20

if not os.path.exists(base_dir) or len(os.listdir(base_dir)) == 0:
    print("Skipping training: Directory not found or empty.")
else:
    def check_class_distribution(base_dir):
        """Check the number of images in each class."""
        class_counts = {cls: len(os.listdir(os.path.join(base_dir, cls))) for cls in os.listdir(base_dir)}
        print("Class distribution:", class_counts)
        return class_counts

    class_counts = check_class_distribution(base_dir)

    def load_data():
        """Load and preprocess training and validation data."""
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=30,
            width_shift_range=0.3,
            height_shift_range=0.3,
            shear_range=0.3,
            zoom_range=0.3,
            horizontal_flip=True,
            brightness_range=[0.8, 1.2],
            validation_split=0.2
        )

        val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # No augmentation for validation

        train_generator = train_datagen.flow_from_directory(
            base_dir,
            target_size=img_size,
            batch_size=batch_size,
            class_mode="categorical",  
            subset="training"
        )

        val_generator = val_datagen.flow_from_directory(
            base_dir,
            target_size=img_size,
            batch_size=batch_size,
            class_mode="categorical",
            subset="validation"
        )

        # Save class mappings
        class_indices = train_generator.class_indices
        with open("class_indices.json", "w") as f:
            json.dump(class_indices, f)
        print("Saved class mappings:", class_indices)

        return train_generator, val_generator

    def compute_class_weights(train_generator):
        """Compute class weights to handle imbalanced data."""
        class_weights = class_weight.compute_class_weight(
            class_weight='balanced',
            classes=np.unique(train_generator.classes),
            y=train_generator.classes
        )
        return dict(enumerate(class_weights))

    def create_model(input_shape, num_classes):
        """Define a CNN using MobileNetV2."""
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
        base_model.trainable = False  # Freeze the base model

        x = GlobalAveragePooling2D()(base_model.output)
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.5)(x)
        output_layer = Dense(num_classes, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=output_layer)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model

    # Load Data
    train_generator, val_generator = load_data()
    num_classes = len(train_generator.class_indices)
    
    # Compute Class Weights
    class_weights = compute_class_weights(train_generator)

    # Build Model
    model = create_model((img_size[0], img_size[1], 3), num_classes)

    # Define Callbacks
    callbacks = [
        EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1),
        ModelCheckpoint("sign_language_best_model1.keras", save_best_only=True, monitor="val_accuracy", mode="max")
    ]

    # Train Model
    model.fit(train_generator, 
              validation_data=val_generator, 
              epochs=epochs, 
              class_weight=class_weights,  
              callbacks=callbacks, 
              steps_per_epoch=len(train_generator),
              validation_steps=len(val_generator))

    # Save Final Model
    model.save("sign_language_model1.keras")
    print("Model training completed and saved as sign_language_model.keras")


ModuleNotFoundError: No module named 'tensorflow'

import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
import json
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load trained model and labels
def load_trained_model():
    """Loads the trained model and the corresponding class labels."""
    try:
        model = load_model("sign_language_model1.keras")
        print("Model loaded successfully.")
    except FileNotFoundError:
        print("Error: Model file not found.")
        return None, None

    try:
        with open("class_indices.json", "r") as f:
            class_indices = json.load(f)
        class_labels = {v: k for k, v in class_indices.items()}  # Reverse mapping
        print("Loaded class labels:", class_labels)
    except FileNotFoundError:
        print("Error: Class label file not found. Using default labels.")
        class_labels = {0: "Unknown", 1: "Unknown"}

    return model, class_labels

# Load model and labels
model, class_labels = load_trained_model()
if model is None:
    exit()

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Start webcam
cap = cv2.VideoCapture(0)

def real_time_prediction(model, class_labels):
    """Perform real-time predictions from webcam input."""
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        h, w, c = frame.shape
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                x_min, y_min, x_max, y_max = w, h, 0, 0
                for lm in hand_landmarks.landmark:
                    x, y = int(lm.x * w), int(lm.y * h)
                    x_min, y_min = min(x, x_min), min(y, y_min)
                    x_max, y_max = max(x_max, x), max(y_max, y)

                padding = 40
                x_min, y_min = max(x_min - padding, 0), max(y_min - padding, 0)
                x_max, y_max = min(x_max + padding, w), min(y_max + padding, h)

                if x_max - x_min > 10 and y_max - y_min > 10:
                    hand_roi = frame[y_min:y_max, x_min:x_max]
                    target_size = model.input_shape[1:3]
                    hand_roi = cv2.resize(hand_roi, target_size)
                    hand_roi = cv2.cvtColor(hand_roi, cv2.COLOR_BGR2RGB)
                    hand_roi = img_to_array(hand_roi) / 255.0  
                    hand_roi = np.expand_dims(hand_roi, axis=0)

                    predictions = model.predict(hand_roi)[0]
                    confidence = np.max(predictions)
                    predicted_class = np.argmax(predictions)
                    predicted_label = class_labels.get(predicted_class, "Unknown")

                    if confidence > 0.7:  # Adjust threshold
                        color = (0, 255, 0)
                    else:
                        predicted_label = "No sign detected"
                        color = (0, 0, 255)

                    cv2.putText(frame, f"Prediction: {predicted_label} ({confidence:.2f})",
                                (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow("ASL Hand Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Perform real-time prediction
real_time_prediction(model, class_labels)

cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
import json
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load trained model and labels
def load_trained_model():
    """Loads the trained model and the corresponding class labels."""
    try:
        model = load_model("sign_language_model1.keras")
        print("Model loaded successfully.")
    except FileNotFoundError:
        print("Error: Model file not found.")
        return None, None

    try:
        with open("class_indices.json", "r") as f:
            class_indices = json.load(f)
        class_labels = {v: k for k, v in class_indices.items()}  # Reverse mapping
        print("Loaded class labels:", class_labels)
    except FileNotFoundError:
        print("Error: Class label file not found. Using default labels.")
        class_labels = {0: "Unknown", 1: "Unknown"}

    return model, class_labels

# Load model and labels
model, class_labels = load_trained_model()
if model is None:
    exit()

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Start webcam
cap = cv2.VideoCapture(0)

def detect_hand_box_and_predict(model, class_labels):
    """Detects and draws a bounding box around the hand, and uses the model for predictions."""
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        h, w, c = frame.shape
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Get bounding box coordinates
                x_min, y_min, x_max, y_max = w, h, 0, 0
                for lm in hand_landmarks.landmark:
                    x, y = int(lm.x * w), int(lm.y * h)
                    x_min, y_min = min(x, x_min), min(y, y_min)
                    x_max, y_max = max(x_max, x), max(y_max, y)

                # Add padding around the hand
                padding = 40
                x_min, y_min = max(x_min - padding, 0), max(y_min - padding, 0)
                x_max, y_max = min(x_max + padding, w), min(y_max + padding, h)

                # Draw the bounding box around the hand
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

                # Extract the hand ROI (Region of Interest) from the frame
                hand_roi = frame[y_min:y_max, x_min:x_max]
                
                # Preprocess the hand ROI for the model
                target_size = model.input_shape[1:3]  # Size your model expects
                hand_roi_resized = cv2.resize(hand_roi, target_size)
                hand_roi_resized = cv2.cvtColor(hand_roi_resized, cv2.COLOR_BGR2RGB)
                hand_roi_resized = img_to_array(hand_roi_resized) / 255.0
                hand_roi_resized = np.expand_dims(hand_roi_resized, axis=0)

                # Predict using the model (optional)
                predictions = model.predict(hand_roi_resized)
                predicted_class = np.argmax(predictions)
                predicted_label = class_labels.get(predicted_class, "Unknown")
                confidence = np.max(predictions)

                # Display prediction label if confidence is high
                if confidence > 0.7:
                    color = (0, 255, 0)  # Green for high confidence
                    cv2.putText(frame, f"Prediction: {predicted_label} ({confidence:.2f})",
                                (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                else:
                    color = (0, 0, 255)  # Red for low confidence
                    cv2.putText(frame, "No clear prediction", (x_min, y_min - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

                # Optionally, you can also draw the hand landmarks
                # mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the frame with the bounding box and prediction
        cv2.imshow("Hand Detection and Prediction", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Perform hand detection, draw bounding box, and prediction
detect_hand_box_and_predict(model, class_labels)

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 